### Scenario
You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

In [1]:
# Import liabraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read that data into Python and call the dataframe churnData

churnData = pd.read_csv('files_for_lab/Customer-Churn.csv')
churnData

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.6,Yes


In [3]:
# Check datatypes

churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
# Convert column `TotalCharges` into numeric type

churnData.TotalCharges = pd.to_numeric(churnData.TotalCharges, errors='coerce')

In [5]:
# Check for null values

churnData.isna().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [6]:
# Replace null values

churnData.TotalCharges.fillna(0, inplace=True)

In [7]:
# Use features `tenure`, `SeniorCitizen`, `MonthlyCharges`, and `TotalCharges`

features = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']
X = churnData[features]
y = churnData['Churn']

In [8]:
# Scale the features using StandardScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# Split the data into training set and test set

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

In [10]:
# Fit a logistic regression model on the training data

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [11]:
# Make predictions on the test data

y_pred = logreg.predict(X_test)

In [12]:
# Check the accuracy on the test data

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8001135718341851


In [13]:
def model(data):
    features = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']
    X = data[features]
    y = data['Churn']
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    return 

In [14]:
# Check for the imbalance

churnData.Churn.value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [15]:
# Upsampling 

category_no = churnData[churnData['Churn'] == 'No']
category_yes = churnData[churnData['Churn'] == 'Yes']

category_yes = category_yes.sample(len(category_no), replace=True)
churnData_upsampling = pd.concat([category_no, category_yes], axis=0)
#shuffling the data
churnData_upsampling = churnData_upsampling.sample(frac=1)
print(churnData_upsampling['Churn'].value_counts())

Yes    5174
No     5174
Name: Churn, dtype: int64


In [16]:
model(data=churnData_upsampling)

Accuracy: 0.7386934673366834


In [17]:
# Downsampling

category_no = churnData[churnData['Churn'] == 'No']
category_yes = churnData[churnData['Churn'] == 'Yes']

category_no = category_no.sample(len(category_yes))
churnData_downsampling = pd.concat([category_no, category_yes], axis=0)
#shuffling the data
churnData_downsampling = churnData_downsampling.sample(frac=1)
churnData_downsampling['Churn'].value_counts()

No     1869
Yes    1869
Name: Churn, dtype: int64

In [18]:
model(data=churnData_downsampling)

Accuracy: 0.7529411764705882
